In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import subprocess
import time

In [ ]:
# --------------- parameters --------------- #
# we have R resources and an ancestral population consisting
# of n_start individuals and we study them for n_gen generations
R = 10
n_start = 1e7
n_gen = 1000000 # number of generations
dt_gen = 10000 # number of generations between two sequencings
D = 1000 # depth of sequencing

# number of mutations in the population per generation
NU_X = 5
NU_alpha = 5

n_simulations = 10 # number of simulations with the same parameters
fast = 1 # if we want faster simulations we have to give up "checkpoints"
         #(sequencing every dt_gen generations) (fast = 1, slow = 0)
    
EPS = [1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3] # small perturbations around the completely symmetric state for beta
S = [1e-7, 10**(-6.5), 1e-6, 10**(-5.5), 1e-5, 10**(-4.5), 1e-4, 10**(-3.5)] # fitness increment in mutation

# let's write the data in one string, which we will pass to the command line arguments
data = str(R) + ' ' + str(n_start) + ' ' + str(n_gen) + ' ' + str(t_gen) + ' ' + str(D) + ' ' + str(NU_X) + ' ' + str(NU_alpha) + ' ' + str(n_simulations)+ ' ' + str(fast) + ' ' + ' '.join([str(x) for x in EPS]) + ' ' + ' '.join([str(x) for x in S])

In [ ]:
tstart = time.time()
subprocess.call('make clean', shell = True)
subprocess.call('make', shell = True)
subprocess.call('./main.exe ' + data, shell = True)

print('Time to run the code =', round(time.time() - tstart),'s')

In [ ]:
# this cell produce a plot of # ecotypes vs scaled fitness mutation rate (with generalist initial strain)

# summary of the parameters we used
print('# resources, R = {}'.format(R))
print('Initial population, n_start = {:0.0e}'.format(n_start))
print('# generations = {:0.0e}'.format(n_gen))
print('Depth of sequencing, D = {:0.0e}'.format(D))
print('# simulations per point = {}'.format(n_simulations))
print()
print('NU_X = {}, NU_alpha = {}'.format(NU_X, NU_alpha))
print('Values for eps = ' + ', '.join(['{:.1e}'.format(x) for x in EPS]))
print('Values for s = ' + ', '.join(['{:.1e}'.format(x) for x in S]))

# plot 
plt.rcParams.update({'font.size': 16})
for eps, s in zip(EPS, S):
    data = np.genfromtxt('./Data/n_species_{:0.1e}_{:0.1e}.txt'.format(eps, s), delimiter=',')
    x, y = data[:, 0], data[:, 1]
    yerr = np.std(y)
    plt.errorbar(np.mean(x), np.mean(y), yerr, marker='o', mfc='b', mec='b', ecolor='b')
plt.xscale('log')
plt.title('# ecotypes from generalist initial strain\n')
plt.xlabel(r'Scaled fitness mutation rate, $U_X  s^2  R / (U_\alpha \epsilon^2)$')
plt.ylabel('# ecotypes, S')

# we write the parameters in the plot (in this way we won't forget which values we used!!)
plt.text(0, -0.4, transform = plt.gcf().transFigure, s = '# resources, R = {}'.format(R)+'\n'
                                                     +'Initial population, n_start = {:0.0e}'.format(n_start)+'\n'
                                                     +'# generations = {:0.0e}'.format(n_gen)+'\n'
                                                     +'Depth of sequencing, D = {:0.0e}'.format(D)+'\n'
                                                     +'# simulations per point = {}'.format(n_simulations)+'\n'
                                                     +'NU_X = {}, NU_alpha = {}'.format(NU_X, NU_alpha)+'\n'
                                                     +'Values for eps = '+', '.join(['{:.1e}'.format(x) for x in EPS])+'\n'
                                                     +'Values for s = '+', '.join(['{:.1e}'.format(x) for x in S]),
         fontdict = {'weight':'light','size':11})

plt.savefig('./Data/n_ecotypes.jpg', bbox_inches = 'tight', dpi = 1000)
plt.show()

In [ ]:
# this cell produce a plot of # ecotypes vs generations

# probabilities of mutations
U_X = NU_X / n_start
U_alpha = NU_alpha / (n_start * R)

# plot
plt.rcParams.update({'font.size': 16})
for j, (eps, s) in enumerate(zip(EPS, S)):
    data = np.genfromtxt('./Data/n_gen_{:0.1e}_{:0.1e}.txt'.format(eps, s), delimiter=',')
    gens, ns = data[:, 0], data[:, 1]

    gens = np.split(gens, n_simulations)
    ns = np.split(ns, n_simulations)
    
    mean_ns = np.zeros(n_points)
    
    for i in range(n_simulations):
        plt.plot(gens[i], ns[i], color = 'C'+str(j), alpha = 0.2)
        mean_ns += np.array(ns[i]) / n_simulations
    
    plt.plot(gens[0], mean_ns, color = 'C'+str(j), label='{:0.0e}'.format(U_X*s**2*R/(U_alpha*eps**2)))

plt.title('Convergence of the number of ecotypes')
plt.xlabel('# generations')
plt.ylabel('# ecotypes, S')
plt.legend(title = 'Scaled fitness mutation rate', fontsize = 10, title_fontsize = 11)

# we write the parameters in the plot (in this way we won't forget which values we used!!)
plt.text(0, -0.4, transform = plt.gcf().transFigure, s = '# resources, R = {}'.format(R)+'\n'
                                             +'Initial population, n_start = {:0.0e}'.format(n_start)+'\n'
                                             +'# generations = {:0.0e}'.format(n_gen)+'\n'
                                             +'Depth of sequencing, D = {:0.0e}'.format(D)+'\n'
                                             +'# simulations per point = {}'.format(n_simulations)+'\n'
                                             +'NU_X = {}, NU_alpha = {}'.format(NU_X, NU_alpha)+'\n'
                                             +'Values for eps = '+', '.join(['{:.1e}'.format(x) for x in EPS])+'\n'
                                             +'Values for s = '+', '.join(['{:.1e}'.format(x) for x in S]),
        fontdict = {'weight':'light','size':11})

plt.savefig('./Data/conv_num_ecotypes.jpg', bbox_inches = 'tight', dpi = 1000)